In [1]:
from models.AlexNet import AlexNet, AlexNetCIFAR10, AlexNetCIFAR10ee1, AlexNetCIFAR10ee2, AlexNetWithExistsCIFAR10

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import datasets, transforms
import torchvision
import matplotlib

import os
import numpy as np
from datetime import datetime as dt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
transform = transforms.ToTensor()

batch_size = 600

train_data   = datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
test_data    = datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
model = AlexNetWithExistsCIFAR10(exit_loss_weights=[20, 5, 1]).to(device)
print(model)

AlexNetWithExistsCIFAR10(
  (backbone): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Sequential(
        (0): Conv2d(96, 256, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Sequential(
        (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (2): Sequential(
      (0): Sequential(
        (0): Conv2d(384, 384, kernel_size=(3, 3), stride

In [ ]:
for i, (X_train, y_train) in enumerate(train_data):
    X_train = X_train.to(device)
#    y_train = y_train.to(device)
    break

print(X_train.shape)
    
x = X_train.view(1,3,32,32)
print(x.shape)
results = model.forward(x)
print(results)


In [7]:
# Train backbone

backbone_params = [
      {'params': model.backbone.parameters()},
      {'params': model.exits[-1].parameters()}
]

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(backbone_params, lr=0.001)

import time
start_time = time.time()

epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    trn_corr = 0
    tst_corr = 0
    test_count = 0
    
    # Run the training batches
    for b, (X_train, y_train) in enumerate(train_loader):
        X_train = X_train.to(device)
        y_train = y_train.to(device)
        b+=1
        
        # Apply the model
        y_pred = model(X_train)[-1]  # we don't flatten X-train here
        loss = criterion(y_pred, y_train)
 
        # Tally the number of correct predictions
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print interim results
        if (b-1)%10 == 0:
            print(f'epoch: {i:2}  batch: {b:4} [{batch_size*b:6}/60000]  loss: {loss.item():10.8f}  \
accuracy train: {trn_corr.item()*100/(batch_size*b):7.3f}%')
        
    train_losses.append(loss)
    train_correct.append(trn_corr)
        
    # Run the testing batches
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):
            X_test = X_test.to(device)
            y_test = y_test.to(device)

            # Apply the model
            y_val = model(X_test)[-1]

            # Tally the number of correct predictions
            predicted = torch.max(y_val.data, 1)[1] 
            tst_corr += (predicted == y_test).sum()
            test_count += len(predicted)
            
    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)
    
    print(f"\nLoss: {loss.item():10.8f} - Accuracy Test: {100*tst_corr/test_count:2.2f}\n")
        
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed  

epoch:  0  batch:    1 [   600/60000]  loss: 0.34898761  accuracy train:  88.333%
epoch:  0  batch:   11 [  6600/60000]  loss: 0.48136407  accuracy train:  82.864%
epoch:  0  batch:   21 [ 12600/60000]  loss: 0.48491627  accuracy train:  83.937%
epoch:  0  batch:   31 [ 18600/60000]  loss: 0.39352533  accuracy train:  84.651%
epoch:  0  batch:   41 [ 24600/60000]  loss: 0.38678420  accuracy train:  85.293%
epoch:  0  batch:   51 [ 30600/60000]  loss: 0.40199876  accuracy train:  85.745%
epoch:  0  batch:   61 [ 36600/60000]  loss: 0.32434300  accuracy train:  85.921%
epoch:  0  batch:   71 [ 42600/60000]  loss: 0.41014165  accuracy train:  86.002%
epoch:  0  batch:   81 [ 48600/60000]  loss: 0.32028916  accuracy train:  86.117%

Loss: 0.67486274 - Accuracy Test: 80.98

epoch:  1  batch:    1 [   600/60000]  loss: 0.34843138  accuracy train:  88.333%
epoch:  1  batch:   11 [  6600/60000]  loss: 0.29306197  accuracy train:  88.455%
epoch:  1  batch:   21 [ 12600/60000]  loss: 0.33086836 

In [8]:
# Train Exits

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

import time
start_time = time.time()

epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

print(model.exit_loss_weights)

for i in range(epochs):
    trn_corr = [0, 0, 0]
    tst_corr = [0, 0, 0]
    test_count = 0
    
    # Run the training batches
    for b, (X_train, y_train) in enumerate(train_loader):
        X_train = X_train.to(device)
        y_train = y_train.to(device)
        b+=1
        
        # Apply the model
        y_pred = model(X_train)  
              
        losses = [weighting * criterion(res, y_train) for weighting, res in zip(model.exit_loss_weights, y_pred)]
          
        # Update parameters
        optimizer.zero_grad()        
        for loss in losses[:-1]:
            loss.backward(retain_graph=True)
        losses[-1].backward()
        optimizer.step()
        
        for exit, y_pred_exit in enumerate(y_pred):   
            predicted = torch.max(y_pred_exit.data, 1)[1]
            batch_corr = (predicted == y_train).sum()
            trn_corr[exit] += batch_corr
        
        # Print interim results
        if (b-1)%10 == 0:
            loss_string = [ f'{loss.item():10.8f}' for loss in losses ]
            accu_string = [ f'{correct.item()*100/(batch_size*b):7.3f}%' for correct in trn_corr ]
            print(f'epoch: {i:2}  batch: {b:4} [{batch_size*b:6}/60000]  losses: {loss_string}  \
accuracy train: {accu_string}')
        
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):
            X_test = X_test.to(device)
            y_test = y_test.to(device)

            # Apply the model
            y_val = model(X_test)

            # Tally the number of correct predictions
            for exit, y_val_exit in enumerate(y_val):
                predicted = torch.max(y_val_exit.data, 1)[1]
                batch_corr = (predicted == y_test).sum()
                tst_corr[exit] += batch_corr            
            
            test_count += len(predicted)
                                  
    losses = [ f'{criterion(y_val_exit, y_test).item():10.8f}' for y_val_exit in y_val ]
    accs   = [ f'{100*tst_corr_exit/test_count:2.2f}' for tst_corr_exit in tst_corr ]
    
    print(f"\nLoss: {losses} - Accuracy Test: {accs}\n")
        
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed  

[20, 5, 1]
epoch:  0  batch:    1 [   600/60000]  losses: ['8.50893593', '0.93021142', '0.24012361']  accuracy train: [' 85.833%', ' 93.333%', ' 91.833%']
epoch:  0  batch:   11 [  6600/60000]  losses: ['11.97224426', '1.82826269', '0.29086778']  accuracy train: [' 78.152%', ' 84.682%', ' 90.106%']
epoch:  0  batch:   21 [ 12600/60000]  losses: ['10.89184380', '1.25134408', '0.20737654']  accuracy train: [' 79.802%', ' 87.452%', ' 90.365%']
epoch:  0  batch:   31 [ 18600/60000]  losses: ['8.69488144', '0.95371747', '0.19286132']  accuracy train: [' 81.027%', ' 89.285%', ' 91.145%']
epoch:  0  batch:   41 [ 24600/60000]  losses: ['8.38115025', '0.88418013', '0.17516689']  accuracy train: [' 81.874%', ' 90.329%', ' 91.780%']
epoch:  0  batch:   51 [ 30600/60000]  losses: ['9.17456436', '0.83370042', '0.18573123']  accuracy train: [' 82.458%', ' 91.029%', ' 92.160%']
epoch:  0  batch:   61 [ 36600/60000]  losses: ['9.36941719', '1.00042641', '0.20607631']  accuracy train: [' 82.967%', ' 9